In [1]:
''' 
语言模型的目标是估计整个序列的联合概率。

P(x1, x2, ..., xT).

在自然语言处理中，这个目标通常通过链式规则（Chain Rule）来实现。链式规则是概率论中的一个基本公式，它指出任何多个事件的联合概率都可以表示为它们的条件概率的乘积。对于文本序列，链式规则可以表示为：

P(x1, x2, ..., xT) = P(x1)P(x2|x1)P(x3|x1,x2)...P(xT|x1,...,xT-1).

这个公式的意思是，一个序列的联合概率等于每个单词（或字符、标记）在给定前面所有单词的条件下的概率的乘积。

在实践中，由于计算和存储所有可能的词序列的条件概率非常困难，因此通常会采用一些简化的假设。例如，n-gram 模型假设每个单词只依赖于前 n-1 个单词，而不是所有的前面的单词。另一种常见的假设是马尔可夫假设，它假设每个单词只依赖于前一个单词。这些假设可以大大简化模型的复杂性，同时仍然能够获得较好的结果。
'''

''' 
尽管简化了的语言模型可能受到一定的限制，但它们在实际应用中仍然具有很大的价值。例如：

1. 语音识别：在语音识别任务中，类似 sounding 的短语可能会导致歧义。如 “to recognize speech” 和 “to wreck a nice beach” 在发音上非常相似。通过使用语言模型，我们可以轻松地解决这种歧义，因为模型会将第二个翻译视为离谱的，从而倾向于选择第一个翻译。

2. 文档摘要：在文档摘要算法中，了解词组的频率分布非常重要。例如，知道 “dog bites man” 比 “man bites dog” 更常见，可以帮助模型更好地理解语料库中的主要话题。同样，了解 “I want to eat grandma” 是一个令人不安的陈述，而 “I want to eat, grandma” 则相对温和，有助于模型更准确地捕捉语义。

通过捕捉词汇和词组在文本中的分布规律，语言模型可以帮助我们解决各种自然语言处理任务中的问题。虽然简化模型可能无法完全捕捉所有的语言结构，但它们在很多情况下已经足够有效。当然，随着深度学习技术的发展，我们可以使用更复杂的模型（如循环神经网络和 Transformer）来捕捉更丰富的语言结构，从而进一步提高自然语言处理任务的性能。
''' 

import torch
from d2l import torch as d2l


在这里，我们假设训练数据集是一个大型文本语料库，例如所有维基百科条目、古腾堡计划以及网上发布的所有文本。单词的概率可以从训练数据集中给定单词的相对词频计算得出。例如，可以将估计 P(x1) 计算为任何以单词 "deep" 开头的句子的概率。一种稍微不那么准确的方法是计算单词 "deep" 的所有出现次数，并将其除以语料库中的总词数。对于频繁出现的单词，这种方法效果相当好。接下来，我们可以尝试估计：

P(x2|x1) = n(x1, x2) / n(x1),

其中 n(x1, x2) 和 n(x1) 分别表示单词和连续单词对的出现次数。不幸的是，估计单词对的概率有点困难，因为 "deep learning" 的出现次数要少得多。特别是对于一些不常见的单词组合，可能很难找到足够的出现次数来获得准确的估计。正如第9.2.5节的实证结果所示，对于三个单词的组合及更多的情况，情况变得更糟。在我们的数据集中，可能有很多看似合理的三个单词的组合，但我们可能找不到。除非我们提供某种解决方案为这样的单词组合分配一个非零计数，否则我们将无法在语言模型中使用它们。如果数据集很小或者单词非常罕见，我们甚至可能找不到其中的任何一个。

为了解决这个问题，我们可以采用一些平滑技术，如拉普拉斯平滑（Laplace smoothing）或古德-图灵估计（Good-Turing estimation），为低频或未出现的 n-gram 分配一个非零概率。这样，即使在训练数据中未见过的词组合也可以在语言模型中得到合理的处理。此外，可以使用更复杂的模型（如循环神经网络和 Transformer）来捕捉更丰富的语言结构，从而进一步提高自然语言处理任务的性能。

基于上述原因，像拉普拉斯平滑这样的简单模型在实际应用中会变得非常笨重：

1. 如第9.2.5节所讨论的，许多 n-gram 出现得非常少，这使得拉普拉斯平滑不太适合用于语言建模。

2. 我们需要存储所有的计数。随着数据集和词汇表的增长，存储和计算需求会变得非常大。

3. 这种方法完全忽略了单词的意义。例如，“cat” 和 “feline” 应该出现在相关的上下文中。调整这样的模型以适应其他上下文非常困难，而基于深度学习的语言模型非常适合解决这个问题。

4. 长的词序列几乎肯定是新颖的，因此仅仅计算先前看到的词序列频率的模型在这方面表现不佳。

因此，在本章的其余部分，我们将专注于使用神经网络进行语言建模。神经网络语言模型可以更好地处理上述问题，例如捕捉词义信息、适应不同上下文以及处理稀疏性问题。此外，神经网络模型可以通过端到端的训练方法学习到更丰富的语言结构，从而在自然语言处理任务中取得更好的性能。

为了衡量语言模型的质量，我们可以计算序列的似然性。然而，这个数字很难理解和比较。毕竟，较短的序列比较长的序列更有可能出现，因此在托尔斯泰的巨著《战争与和平》上评估模型将不可避免地产生比在圣埃克苏珀里的小说《小王子》上更小的似然性。我们需要类似平均值的衡量标准。

信息论在这里非常有用。当我们介绍 softmax 回归时（第4.1.3节），我们定义了熵、惊讶度和交叉熵。如果我们想压缩文本，我们可以询问在给定当前一组标记的情况下预测下一个标记。更好的语言模型应该使我们能够更准确地预测下一个标记。因此，它应该使我们在压缩序列时使用更少的比特。因此，我们可以通过计算序列中所有 T 个标记上的交叉熵损失的平均值来衡量模型的质量：

L = - (1/T) * Σ [y_t * log(y_t^hat)],

其中 y_t 是实际标记的 one-hot 编码，y_t^hat 是模型对下一个标记的预测概率分布。这个损失值越小，说明模型的预测越准确，因此语言模型的质量越高。

在实际应用中，我们可以使用这个平均交叉熵损失来评估和比较不同的语言模型。一个好的语言模型应该能够在各种文本序列上产生较低的平均交叉熵损失，这意味着模型能够更准确地预测和理解自然语言的结构和模式。

困惑度（perplexity）是自然语言处理领域中常用的评价指标，它可以看作是交叉熵损失的指数：

perplexity = exp(L),

其中 L 是平均交叉熵损失。困惑度可以最好地理解为在决定选择下一个标记时，我们实际选择的几何平均数的倒数。让我们看几个例子：

1. 最佳情况：模型总是完美地估计目标标记的概率为1。在这种情况下，模型的困惑度为1。

2. 最坏情况：模型总是预测目标标记的概率为0。在这种情况下，困惑度为正无穷。

3. 基准情况：模型预测词汇表中所有可用标记的均匀分布。在这种情况下，困惑度等于词汇表中唯一标记的数量。实际上，如果我们要在不进行任何压缩的情况下存储序列，这将是对其进行编码的最佳方法。因此，这为任何有用的模型提供了一个非平凡的上界，必须击败这个上界。

在实际应用中，我们可以使用困惑度来评估和比较不同的语言模型。较低的困惑度表示模型对文本序列的预测更准确，从而更好地捕捉自然语言的结构和模式。通过优化模型结构和训练方法，我们可以降低困惑度，从而提高自然语言处理任务的性能。

困惑度（perplexity）是衡量语言模型预测能力的指标。它的定义是交叉熵损失的指数：

perplexity = exp(L),

其中 L 是平均交叉熵损失。当困惑度为1时，这意味着平均交叉熵损失 L 为0。交叉熵损失的计算公式如下：

L = -(1/T) * Σ [y_t * log(y_t^hat)],

其中 y_t 是实际标记的 one-hot 编码，y_t^hat 是模型对下一个标记的预测概率分布，T 是序列长度。

当交叉熵损失为0时，这意味着 y_t * log(y_t^hat) 对于所有的 t 都为0。由于 y_t 是 one-hot 编码，对于正确的目标标记，y_t = 1，而对于其他标记，y_t = 0。要使 y_t * log(y_t^hat) 为0，正确的目标标记的预测概率 y_t^hat 必须为1（因为 log(1) = 0）。这意味着模型总是完美地预测下一个标记，没有任何错误，因此困惑度为1。

当困惑度为正无穷时，这意味着模型总是预测目标标记的概率为0，即 y_t^hat = 0。在这种情况下，交叉熵损失会变得非常大（因为 log(0) 趋向于负无穷），因此困惑度也会变为正无穷。这表示模型的预测能力非常差，完全无法预测正确的目标标记。

在这里，“非平凡”（nontrivial）意味着一个有意义的、实际上可以实现的上界。当模型预测词汇表中所有可用标记的均匀分布时，困惑度等于词汇表中唯一标记的数量。这个上界是非平凡的，因为它实际上可以达到（当我们没有任何关于文本结构的信息时，我们只能做出一个均匀的猜测），并且它比最差情况（困惑度为正无穷）要好得多。

因此，任何有用的语言模型都应该在这个非平凡的上界之下，即其困惑度应该低于词汇表中唯一标记的数量。通过改进模型结构和训练方法，我们可以降低困惑度，从而更好地捕捉自然语言的结构和模式，并在自然语言处理任务中取得更好的性能。

在设计基于神经网络的语言模型时，我们将使用困惑度来评估模型在给定文本序列中当前一组标记的情况下预测下一个标记的能力。在介绍模型之前，让我们假设它一次处理一个具有预定义长度的序列的小批量。现在的问题是如何随机读取输入序列和目标序列的小批量。

假设数据集采用语料库中的一系列 T 个标记索引的形式。我们将其划分为子序列，每个子序列具有 n 个标记（时间步）。为了在每个周期中迭代整个数据集的（几乎）所有标记并获得所有可能的长度为 n 的子序列，我们可以引入随机性。更具体地说，在每个周期开始时，丢弃前 s 个标记，其中 s 是随机均匀采样的。然后将剩余的序列划分为 T-n+1 个子序列。用 x_{i} 表示从时间步 i 开始的长度为 n 的子序列。得到的 T-n+1 个划分子序列是 x_1, x_2, ..., x_{T-n+1}。每个子序列将作为输入序列输入到语言模型中。

对于语言建模，目标是基于迄今为止看到的标记来预测下一个标记；因此，目标（标签）是原始序列，向后移动一个标记。任何输入序列 x_i 的目标序列是 y_i，长度为 n。

![lang-model-data](https://d2l.ai/_images/lang-model-data.svg)

图 9.3.1 从划分的长度为5的子序列中获取五对输入序列和目标序列的示例。

图 9.3.1 显示了 n=5 和 T=20 的情况下，获取五对输入序列和目标序列的示例。这种方法可以确保在训练过程中，我们随机地从数据集中获取不同的输入序列和目标序列，从而提高模型的泛化能力和预测性能。

In [2]:
@d2l.add_to_class(d2l.TimeMachine)  # @save
def __init__(self, batch_size, num_steps, num_train=10000, num_val=5000):
    super(d2l.TimeMachine, self).__init__()
    self.save_hyperparameters()
    corpus, self.vocab = self.build(self._download())
    array = torch.tensor([corpus[i:i+num_steps+1]
                          for i in range(len(corpus)-num_steps)])
    self.X, self.Y = array[:, :-1], array[:,1:]

In [3]:
@d2l.add_to_class(d2l.TimeMachine)  # @save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(
        self.num_train, self.num_train + self.num_val)
    return self.get_tensorloader([self.X, self.Y], train, idx)

In [4]:
data = d2l.TimeMachine(batch_size=2, num_steps=10)
for X, Y in data.train_dataloader():
    print('X:', X, '\nY:', Y)
    break

X: tensor([[ 2,  4,  6,  0, 21,  9,  6,  0, 21, 10],
        [10, 21,  9,  0,  2,  0, 13,  6,  2, 15]]) 
Y: tensor([[ 4,  6,  0, 21,  9,  6,  0, 21, 10, 14],
        [21,  9,  0,  2,  0, 13,  6,  2, 15,  0]])


Language models are essential tools in natural language processing and understanding. They aim to predict the probability of a sequence of words appearing in a text, which is beneficial for various applications such as machine translation, speech recognition, and text generation.

N-grams are a popular method for language modeling, where the dependence of words in a sequence is truncated to a fixed length (n words). However, n-grams may not be efficient for dealing with infrequent word combinations, as they rely on the frequency of observed word sequences. Laplace smoothing can help to some extent, but it may not be sufficient for capturing the complex structure of language.

Neural language models have emerged as a powerful alternative to n-grams, as they can capture long-range dependencies and generalize better to unseen word combinations. These models are trained on large datasets by sampling input and target sequences in minibatches. The quality of a trained language model can be assessed using perplexity, which measures how well the model predicts the probability distribution of the words in the text.

As neural language models grow in size and are trained on more extensive data with increased computational resources, they become more capable of performing complex tasks. Large language models can understand and generate text based on input instructions, making them the foundation of state-of-the-art systems in various NLP tasks, such as machine translation, question-answering, and text summarization.

In summary, neural language models offer a powerful approach to capturing the complex structure and dependencies in natural language. By scaling up model size, data size, and training compute, these models can achieve remarkable performance across a wide range of NLP tasks.

1. In a four-gram model, you need to store the frequency of every possible combination of 4 words. Assuming there are 100,000 unique words in the training dataset, you would need to store the frequency of 100,000^4 = 1e20 (100 trillion) possible four-word combinations. However, in practice, not all combinations will be observed in the dataset, and the actual storage requirement will be lower.

2. To model a dialogue, you can use a sequence-to-sequence (seq2seq) model with attention mechanisms. The model typically consists of an encoder that processes the input (e.g., a question or a statement) and a decoder that generates the output (e.g., a response). The attention mechanism helps the model to focus on specific parts of the input while generating the output, making it more context-aware and better at handling long sequences.

3. Other methods for reading long sequence data include:
   - Hierarchical models: Break down the sequence into smaller chunks and process them separately before combining the results.
   - Transformer models: Use self-attention mechanisms to process input sequences in parallel, capturing long-range dependencies more effectively.
   - Sliding window approach: Process the long sequence in smaller, overlapping windows and combine the results.

4. Discarding a uniformly random number of the first few tokens at the beginning of each epoch does not guarantee a perfectly uniform distribution over the sequences in the document. It only ensures that the starting positions of the sequences are uniformly distributed.

5. To make things even more uniform, you could:
   - Shuffle the entire document at the beginning of each epoch.
   - Randomly select starting positions for each sequence in the minibatch, ensuring they are uniformly distributed across the document.

6. If we want a sequence example to be a complete sentence, this introduces a problem in minibatch sampling as sentences have varying lengths. This can lead to inefficient minibatch processing due to the need for padding and masking.

   To fix this, you can:
   - Bucketing: Group sentences of similar lengths together and create minibatches from these groups. This minimizes the amount of padding required.
   - Truncate or pad sentences to a fixed length: This ensures that all sequences in a minibatch have the same length, but it may result in loss of information (truncation) or increased computation (padding).